rank\
nationality\
driver (name)\
Team (constructor name)\
Wins\
Points

In [0]:
dbutils.widgets.text('p_file_date', "2021-03-21")
v_file_date = dbutils.widgets.get('p_file_date')

In [0]:
%run ../includes/configuration

In [0]:
%run ../includes/common_functions

In [0]:
from pyspark.sql.functions import col, count, sum, when, desc, rank
from pyspark.sql.window import Window

Find race years for which data should be processed

In [0]:
race_results_list = spark.read.format("delta").load(f"{presentation_folder_path}/f1_presentation/race_results")\
    .filter(f"file_date = '{v_file_date}'")

In [0]:
race_year_list = df_col_to_list(race_results_list, 'race_year')

In [0]:
race_results_df = spark.read.format("delta").load(f"{presentation_folder_path}/f1_presentation/race_results")\
    .filter(col("race_year").isin(race_year_list))

In [0]:
driver_standings_df = race_results_df\
    .groupBy("race_year", "driver_nationality", "driver_name")\
        .agg(sum("points").alias("total_Points")\
            ,count(when(col("position") == 1, True)).alias("Wins"))

In [0]:
driverRankSpec = Window.partitionBy("race_year").orderBy(desc("total_Points"), desc("wins"))

In [0]:
final_df = driver_standings_df\
    .withColumn("Rank", rank().over(driverRankSpec))

In [0]:
#final_df.write\
#    .mode("overwrite")\
#        .parquet(f"{presentation_folder_path}/driver_standings")

#final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.driver_standings")


#incremental_load(final_df, 'race_year', 'f1_presentation', 'driver_standings')

In [0]:
merge_condition = "tgt.race_year == src.race_year and tgt.driver_name = src.driver_name"

merge_delta_table(final_df, 'f1_presentation', 'driver_standings', presentation_folder_path, merge_condition, 'race_year')

In [0]:
%sql
SELECT race_year, COUNT(race_year)
FROM f1_presentation.driver_standings
GROUP BY race_year
ORDER BY race_year DESC;

race_year,count(race_year)
2021,20
2020,23
2019,20
2018,20
2017,25
2016,24
2015,22
2014,24
2013,23
2012,25
